<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px" />

# Excel Madness Lab!

_Author:_ Tim Book

## Our Mission
We work for a large supermarket chain, with stores in 10 major cities that happen to coincide with General Assembly campuses. However, this company's idea of a "database" is just a bunch of Excel spreadsheets! In order to analyze our data, we're going to need to process the existing data into a form we can use. **Our end goal is to have one csv per city.**

## Cleanup Duty!
It is a hard truth that data scientists spend a large majority of their time cleaning data. Data never arrives on our desks in exactly the format in which we want it, and it's up to us to transform it to a workable format.

Being good cleaning, moving, and reshaping data is in itself a valuable and employable job skill. If you follow these directions exactly, we will walk through constructing an automated process for processing data from this supermarket chain.

# Part I: Processing

### Step 1: Imports and the `os` library
We're going to import three libraries: numpy, pandas, and `os`.

In [1]:
# Import libraries here.
import pandas as pd
import numpy as np
import os

The `os` library is extremely useful for performing system commands from within Python. Let's get two pieces of overhead out of the way now:

1. Create an `output` folder using `os.mkdir()`
2. Create a variable called `files` that is the list of files in the `data` folder using `os.listdir()`

**WARNING:** The `os.mkdir()` function will give you an error if you try to make a folder that already exists!

In [2]:
# Create an output folder.

os.mkdir('output')

# Create a files variable that contains all of our data files.
files = os.listdir('data/')

In [3]:
files

['Jan 23.xlsx',
 'Jan 19.xlsx',
 'Jan 15.xlsx',
 'Jan 7.xlsx',
 'Jan 6.xlsx',
 '~$Jan 1.xlsx',
 'Jan 14.xlsx',
 'Jan 18.xlsx',
 'Jan 22.xlsx',
 'Jan 29.xlsx',
 'Jan 13.xlsx',
 'Jan 1.xlsx',
 'Jan 25.xlsx',
 'Jan 24.xlsx',
 'Jan 12.xlsx',
 'Jan 28.xlsx',
 'Jan 3.xlsx',
 'Jan 11.xlsx',
 'Jan 27.xlsx',
 'Jan 31.xlsx',
 'Jan 30.xlsx',
 'Jan 26.xlsx',
 'Jan 10.xlsx',
 'Jan 2.xlsx',
 'Jan 9.xlsx',
 'Jan 21.xlsx',
 'Jan 5.xlsx',
 'Jan 17.xlsx',
 'Jan 16.xlsx',
 'Jan 4.xlsx',
 'Jan 20.xlsx',
 'Jan 8.xlsx']

### Step 2: Process one data frame
It looks like we have data for the month of January. 31 files of 10 sheets each! Luckily they are all in the same format. So let's read just one in and process that. It might be helpful to open one up in your spreadsheet viewer of choice first (Excel, Numbers, Sheets, etc.)

In [4]:
excel_file = pd.ExcelFile('data/Jan 1.xlsx')
excel_file.sheet_names

['New York',
 'Boston',
 'Washington, DC',
 'Atlanta',
 'Chicago',
 'Denver',
 'Austin',
 'Seattle',
 'Los Angeles',
 'San Francisco']

In [5]:
# Read in data from your city from January 1st.
data = pd.read_excel('data/Jan 1.xlsx',sheet_name='New York')

In [6]:
data.head(3)

,prodcode,price_eu,weight_kg,quantity
0,4159,2.048141,8.812961,138
1,4062,2.728485,9.331372,184
2,4404,2.182498,8.937798,491


### Step 2a: Convert to 'Merican columns
For whatever reason, our data are stored in euros and kilograms. Create `price_usd` and `weight_lb` columns. There are 2.2 pounds per kilogram, and 1.1 dollars per euro.

In [7]:
data['price_usd'] = data['price_eu'] * 1.1
data['weight_lb'] = data['weight_kg'] * 2.2

### Step 2b: Merge in product names
You'll notice we also have a `plu-codes.csv` file containing actual product names matched up against their price lookup (PLU) codes. Let's merge these product names onto our Jan 1 data.
* _Hint 1:_ What kind of merge is this? Right, left, inner, outer, etc.?
* _Hint 2:_ Pay special attention to column names!

In [8]:
plu = pd.read_csv("plu-codes.csv")

In [9]:
data = data.merge(plu,how='left',left_on='prodcode',right_on='plu_code')

### Step 2c: Drop unnecessary columns
We've created some extraneous columns. Drop the old price and weight columns, as well as any redundant columns.

In [10]:
data = data.drop(['price_eu','weight_kg','plu_code'], axis=1)

### Step 2d: Add the date
Simply create a new `date` column that is the date this data was collected. For example, if this is from `Jan 1.xlsx`, this column should be full of `Jan 1`.

In [11]:
data['date'] = 'Jan 1'

### Step 3: Write a function that conducts all of Step 2
This function should import a **filename and a city name** and return a fully processed DataFrame. That is, the function should:
1. Read in the data from the given file and city.
1. Create USD and pound columns.
1. Merge in product names.
1. Drop unnecessary columns.
1. Add a date column

In [12]:
def process_data(file, city):
    
    file_path = 'data/'+file
    date = file.split('.')[0]
    
    data = pd.read_excel(file_path, sheet_name=city)
    
    data['price_usd'] = data['price_eu'] * 1.1
    data['weight_lb'] = data['weight_kg'] * 2.2
    data = data.merge(plu,how='left',left_on='prodcode',right_on='plu_code')
    
    data = data.drop(['price_eu','weight_kg','plu_code'], axis=1)
    
    data['date'] = date
    
    return data
    

Test your function out on a new file and city!

In [13]:
test = process_data('Jan 2.xlsx', 'Boston')

In [14]:
test.head(3)

,prodcode,quantity,price_usd,weight_lb,product,date
0,4030,109,1.974055,16.298007,Kiwi,Jan 2
1,4796,329,2.712247,10.411764,Tomato,Jan 2
2,4309,202,2.601073,5.135373,Lychee,Jan 2


### Step 4: Process all of January's data
For each spreadsheet, process the data and store the resulting DataFrame in one big list. **You only need to do this for your city!**

* _Hint 1:_ A listcomp would make this whole step one line of code!
* _Hint 2:_ You've already made that `files` variable to help you here.

In [15]:
df_list = [process_data(filename, 'New York') for filename in files ]
# I don't know why there is a file called ~$Jan 1.xlsx exist so I need to exclude it.

### Step 5: Concatenate all DataFrames from Step 4 into one large DataFrame
* _Hint:_ Is there a function in `pandas` that can do something like this for us? This is also just one line of code!

In [16]:
files

['Jan 23.xlsx',
 'Jan 19.xlsx',
 'Jan 15.xlsx',
 'Jan 7.xlsx',
 'Jan 6.xlsx',
 '~$Jan 1.xlsx',
 'Jan 14.xlsx',
 'Jan 18.xlsx',
 'Jan 22.xlsx',
 'Jan 29.xlsx',
 'Jan 13.xlsx',
 'Jan 1.xlsx',
 'Jan 25.xlsx',
 'Jan 24.xlsx',
 'Jan 12.xlsx',
 'Jan 28.xlsx',
 'Jan 3.xlsx',
 'Jan 11.xlsx',
 'Jan 27.xlsx',
 'Jan 31.xlsx',
 'Jan 30.xlsx',
 'Jan 26.xlsx',
 'Jan 10.xlsx',
 'Jan 2.xlsx',
 'Jan 9.xlsx',
 'Jan 21.xlsx',
 'Jan 5.xlsx',
 'Jan 17.xlsx',
 'Jan 16.xlsx',
 'Jan 4.xlsx',
 'Jan 20.xlsx',
 'Jan 8.xlsx']

In [17]:
df_concat = pd.concat(df_list,ignore_index=True)

In [22]:
df_concat.head(3)

,prodcode,quantity,price_usd,weight_lb,product,date
0,4650,109,1.330281,17.027600,Mushroom,Jan 23
1,4011,206,1.857675,7.891258,Banana,Jan 23
2,4796,416,2.666518,20.348500,Tomato,Jan 23


### Step 6: Do this for all cities, write data
Here's the big one. For each city, process and the data as in steps 3-5, and then write the data to our `output` folder. Below is a dictionary of city name to desired output file name.

Before writing your DataFrame, do the following:
* Add a `city` column
* Reorder the columns into the following order:


| city | date | product | prodcode | quantity | weight_lb | price_usd |
|---|---|---|---|---|---|---|

* _Hint:_ You can reorder DataFrame columns simply by writing over your DataFrame with itself, but specifying the specific column order with `.loc`. For example:

`print(df)`

| b | c | a |
|---|---|---|
| 1 | 2 | 3 |

`df = df.loc[:, ["a", "b", "c"]]`

`print(df)`

| a | b | c |
|---|---|---|
| 3 | 1 | 2 |


In [19]:
city_dict = {
    "Atlanta": "atl.csv",
    "Austin": "atx.csv",
    "Boston": "bos.csv",
    "Chicago": "chi.csv",
    "Denver": "den.csv",
    "Los Angeles": "lax.csv",
    "New York": "nyc.csv",
    "San Francisco": "sf.csv",
    "Seattle": "sea.csv",
    "Washington, DC": "dc.csv"
}

In [23]:
# Loop through city_dict and carry out Step 6 here.
# The keys of city_dict can serve as the sheet name.
# The values of city_dict are what you should name the output .csv files.
# If done correctly, this cell could take almost a minute to run!

for city in city_dict:
    file_name = city_dict[city]
    final_path = 'output/'+file_name
    
    list_df = [process_data(filename, city) for filename in files if not filename.startswith('~')]
    concat_df = pd.concat(list_df,ignore_index=True)
    concat_df['city'] = city
    
    concat_df = concat_df.loc[:,['city','date','product','prodcode','quantity','weight_lb','price_usd']]
    
    concat_df.to_csv(final_path,index=False)
    

# Part II: Checking our answers 
In steps very similar to the ones conducted above...
1. Loop through the files we just wrote to `output`, and read them in, collecting them all in one list
1. Combine all of those DataFrames into one large DataFrame
1. For each city, find the mean `quantity`, `weight_lb`, and `price_usd`.

If you've done everything correct, your answer should look exactly like this:

![](imgs/correct-output.png)

In [ ]:
output_files = os.listdir('output/')

In [ ]:
output_files

In [ ]:
all_city = pd.concat([pd.read_csv(f'output/{filename}') for filename in output_files if not filename.startswith('.')],ignore_index=True)
# there is a hidden file was created automatically, so I need to ignore it.

In [ ]:
all_city.groupby('city')['quantity','weight_lb','price_usd'].mean()

# Part III (BONUS): Get this process production-ready!
_This part of the lab is optional, but very highly recommended, as the skills developed in this part are extremely common in industry._

For this step, we're going to take this whole process and put it into a production-ready Python script. **ABSOLUTELY NONE OF THIS STEP SHOULD TAKE PLACE IN A JUPYTER NOTEBOOK! PRODUCTIONALIZED ETL (_"Extract, Transform, Load"_) CODE DOES NOT TAKE PLACE IN A JUPYTER NOTEBOOK!**

The instructions are simple: As conducted in this lab, read, transform, and export the data in our Excel files into .csv files. The code should be in a `.py` file and executable from the command line. Here are some hints and tips to guide you:

### Hints, tips, and tricks:
* A good place to start is with the code you've already written. In this notebook, you can click `File > Download as > Python (.py)` to export as a `.py` file. Most of this exercise then comes down to you cleaning this file. (There will be a lot to clean).
* Remember `os.mkdir()` will throw an error if the folder you're trying to make already exists. Maybe you should check to see if it already exists. If it already exists, what should you do? (Remember that `.csv` can be overwritten with no problem.) The functions that can help you with this are all in the `os` library.
* Remember to follow all of the Python best practices we've already learned:
    - All import statements should go at the top of your script.
    - Comment your code. Comments shouldn't explain _what_ code does, but _why_ the code does this.
    - Keep your code DRY (don't repeat yourself) as opposed to WET (write everything twice). All constants should be variables that only need to be changed once. All code should be bottled into functions so you only need to fix it once.
* Make sure not to hardcode "Jan" anywhere. The point is that this code can be run throughout the lifetime of this supermarket's project, which is likely months or years. Keep your code so that if you get February data, you only need to change one tiny piece of the script (probably a file path).